In [ ]:
from dotenv import load_dotenv
import os 
load_dotenv()

True

In [2]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    temperature=0,
    model_name="llama3-70b-8192"
)

In [ ]:
from langchain.prompts import PromptTemplate

template = PromptTemplate(
    input_variables=["input","context"],
    template='''

[INST]

You are a Virtual Assistant for students enrolled in the IITM BS Degree Program. Your primary role is to **answer queries related to the grading structure of any subject** in the program.  

### Guidelines for Answering Grading-Related Queries:
- The student will ask questions about the **grading structure**, including weightage of assignments, exams, quizzes, and overall evaluation criteria.
- Use the **provided context** to give direct and accurate answers.
- **DO NOT use the Socratic Method** or ask guiding questions for grading-related queries.
- If **grading information is missing** from the context, politely inform the student that the details are unavailable and suggest checking official sources.

---

### Closing Statement
**Always provide a clear, factual answer and invite further clarification:**  
**“I hope this helps! Let me know if you need any more details. 😊”**  

---

**Student:** "{input}"  
**Context to answer query:** "{context}"  

[INST]

'''
)

In [62]:
llm_chain = template | llm

In [157]:
from langchain_together import Together
llm2 = Together(
    temperature=0,
    model="meta-llama/Llama-3.3-70B-Instruct-Turbo"
)

/home/asim/Documents/gitRepositories/soft-engg-project-jan-2025-se-Jan-21/backend/.venv/lib/python3.12/site-packages/langchain_together/llms.py:89: UserWarning: The completions endpoint, has 'max_tokens' as required argument. The default value is being set to 200 Consider setting this value, when initializing LLM
  warnings.warn(


>Load the Grading document

In [24]:
# Loading Pinecone vector DB

from pinecone import Pinecone , ServerlessSpec
pinecone_api_key = os.getenv("PINECONE_API_KEY")
index_name="test-rag"

pc_db= Pinecone(api_key=pinecone_api_key)

In [ ]:
if not pc_db.has_index(index_name):
    pc_db.create_index(
        
    )

In [7]:
from langchain.document_loaders import PyPDFLoader

loader =   PyPDFLoader("assets/BS-grading_doc-Jan2025.pdf")
pages = loader.load()
page = pages[0]


In [8]:
# Merging the pages
from langchain.schema import Document

merged_pages = "/n".join([page.page_content for page in pages])
doc = Document(merged_pages)


In [132]:
from langchain.text_splitter import RecursiveCharacterTextSplitter

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=512,
    chunk_overlap=126
)

split_docs = text_splitter.split_documents([doc])


In [133]:
# add the TOGETHER_API_KEY in the .env file
from langchain_together import TogetherEmbeddings
embedding_model = TogetherEmbeddings()


In [134]:
ex = embedding_model.embed_query("What  are you dong")
len(ex)

768

In [135]:
from langchain.vectorstores import Chroma
vector_store2 = Chroma.from_documents(split_docs,embedding_model , persist_directory="new_store2")

In [136]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
qa_chain = create_stuff_documents_chain(llm , template)
RAG = create_retrieval_chain(vector_store2.as_retriever(), combine_docs_chain=qa_chain)

In [163]:
print(RAG.invoke({"input":"Who are you?"})["answer"])

Hello! I'm your Virtual Assistant for the IITM BS Degree Program. I'm here to help answer your queries related to the grading structure of any subject in the program. 😊


In [149]:

response = RAG.invoke({"input":"What does final grade consist of in the software engineering course "})

In [150]:
print(response["answer"])

The final grade in the software engineering course consists of several components: 

* 0.05GAA (Grade Point Average)
* 0.2Qz2 (Quiz 2 score)
* 0.4F (Final exam score)
* 0.1GP1 (Group Project 1 score)
* 0.1GP2 (Group Project 2 score)
* 0.1PP (Peer Performance score)
* 0.05CP (Class Participation score)
* 0.4max(Qz1,Qz2) (Maximum of Quiz 1 and Quiz 2 scores)

I hope this helps! Let me know if you need any more details. 😊


In [155]:
print(RAG.invoke({"input":"What does final grade consist of in the software testing course"})["answer"])

According to the provided context, the final grade in the software testing course consists of:

* 0.4max(Qz1,Qz2) (quiz scores)
* End Term exam (in-person, 30 marks)

Note that the W12 assignment score is not included in the final grade, but its contents will be included in the final exam.

I hope this helps! Let me know if you need any more details. 😊


In [158]:
#! Rough work

In [159]:
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain.chains import create_retrieval_chain
qa_chain2 = create_stuff_documents_chain(llm2 , template)
RAG2 = create_retrieval_chain(vector_store2.as_retriever(), combine_docs_chain=qa_chain)

In [160]:
print(RAG2.invoke({"input":"What does final grade consist of in the software testing course"})["answer"])

According to the provided context, the final grade in the software testing course consists of:

* 0.4max(Qz1,Qz2) (quiz scores)
* End Term exam (in-person, 30 marks)

Note that the W12 assignment score is not included in the final grade, but its contents will be included in the final exam.

I hope this helps! Let me know if you need any more details. 😊
